In [1]:
import torch
import math
import time
import pandas as pd
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader

In [2]:
'''
df = pd.read_json('test.json')
dataset = []
for idx, data in enumerate(df.iloc):
    new_data = []
    new_data.append(data['id'])
    new_data.append(len(data['ingredients']))
    for ingredient in ingredients:
        new_data.append(1 if ingredient in data['ingredients'] else 0)
    dataset.append(new_data)
dataset = pd.DataFrame(dataset)
dataset.columns = ['id', 'num_ingredient'] + [i for i in range(len(ingredients))]
'''
# dataset.to_csv('test.csv', index=False)

"\ndf = pd.read_json('test.json')\ndataset = []\nfor idx, data in enumerate(df.iloc):\n    new_data = []\n    new_data.append(data['id'])\n    new_data.append(len(data['ingredients']))\n    for ingredient in ingredients:\n        new_data.append(1 if ingredient in data['ingredients'] else 0)\n    dataset.append(new_data)\ndataset = pd.DataFrame(dataset)\ndataset.columns = ['id', 'num_ingredient'] + [i for i in range(len(ingredients))]\n"

In [3]:
df = pd.read_csv('./data/test.csv')
ingredients = pd.read_csv('./data/ingredients.csv')['0'].tolist()
cuisines = pd.read_csv('./data/cuisines.csv')['0'].tolist()
display(df)

,id,num_ingredient,0,1,2,3,4,5,6,7,...,6203,6204,6205,6206,6207,6208,6209,6210,6211,6212
0,14,12,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,22,18,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,24,9,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,37,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,40,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,49692,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,49693,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,49697,9,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9998,49709,7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
class MyDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
    def __len__(self):
        return len(self.X)
    def __getitem__(self, index):
        x = self.X[index]
        y = self.Y[index]
        return x, y
    
class DNN_model(nn.Module):
    def __init__(self):
        super(DNN_model, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(6214, 1024),  
            nn.ReLU(),
            nn.BatchNorm1d(1024),
            torch.nn.Dropout(0.7),
            nn.Linear(1024, 1024),  
            nn.ReLU(),
            nn.BatchNorm1d(1024),
            torch.nn.Dropout(0.7),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.BatchNorm1d(1024),
            torch.nn.Dropout(0.7), 
            nn.Linear(1024, 20)
        )

    def forward(self, x):
        return self.fc(x)

model = DNN_model().cuda()

summary(model, (6214, ))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 1024]       6,364,160
              ReLU-2                 [-1, 1024]               0
       BatchNorm1d-3                 [-1, 1024]           2,048
           Dropout-4                 [-1, 1024]               0
            Linear-5                 [-1, 1024]       1,049,600
              ReLU-6                 [-1, 1024]               0
       BatchNorm1d-7                 [-1, 1024]           2,048
           Dropout-8                 [-1, 1024]               0
            Linear-9                 [-1, 1024]       1,049,600
             ReLU-10                 [-1, 1024]               0
      BatchNorm1d-11                 [-1, 1024]           2,048
          Dropout-12                 [-1, 1024]               0
           Linear-13                   [-1, 20]          20,500
Total params: 8,490,004
Trainable param

In [16]:
model = torch.load('./model/model2.pt').cuda()
test_x = torch.tensor(np.array(df.iloc[:,1:]).astype(np.float32)).cuda()
# test_x[:, 0] = (test_x[:, 0] - test_x[:, 0].mean())/(test_x[:, 0].std())
y_pred = model(test_x)
y_pred = np.argmax(y_pred.cpu().data.numpy(), axis=1)
ans = pd.DataFrame([[df['id'][idx], cuisines[y_pred[idx]]] for idx in range(10000)])
ans.columns = ['id', 'Category']
ans.to_csv('y_test.csv', index=False)